In [ ]:
"""
Created on Wed Apr 15 09:53 2022

Apply model coming from train_DNN_with_prepared_input.ipynb on one other run

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import glob
import matplotlib as mpl
import seaborn as sns
import datetime
import time
import os,sys

import tensorflow as tf
from tensorflow import keras
from contextlib import redirect_stdout

from basal_melt_neural_networks.constants import *
import basal_melt_neural_networks.diagnostic_functions as diag
import basal_melt_neural_networks.data_formatting as dfmt

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
outputpath_nn_models = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/'
outputpath_doc = '/bettik/burgardc/SCRIPTS/basal_melt_neural_networks/custom_doc/'
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'
inputpath_data = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/'

In [ ]:
timetag = '20220422-1055'
new_path_output = outputpath_melt_nn+timetag+'/'
if not os.path.isdir(new_path_output):
    print("I did not find this folder "+timetag+") so I created a new one, I hope that's ok!")
    os.mkdir(new_path_output)
else:
    print("This folder ("+timetag+") exists already!")

In [ ]:
nemo_run = 'OPM031'
new_path_model = outputpath_nn_models+timetag+'/'
df_nrun = pd.read_csv( inputpath_data + 'dataframe_input_'+nemo_run+'.csv',index_col=[0,1,2])
all_input_df = df_nrun.copy()
all_input_df.reset_index(drop=True, inplace=True)

PREPARE DATA

In [ ]:
normalisation_coeff = pd.read_csv(new_path_model+ 'dataframe_norm_training_data_'+timetag+'.csv').set_index('Unnamed: 0')#.drop('Unnamed: 0', 1)
normalisation_coeff.index.name = None
normalisation_coeff_input = normalisation_coeff.drop(['melt_m_ice_per_y'], axis=1)

In [ ]:
input_var = all_input_df.drop(['melt_m_ice_per_y'], axis=1)
ref_melt = all_input_df['melt_m_ice_per_y']

In [ ]:
normalised_input_var = (input_var - normalisation_coeff_input.loc['x_mean'])/normalisation_coeff_input.loc['x_range']

In [ ]:
x_val_arr = np.array(normalised_input_var)
y_val_arr = np.array(ref_melt)

CHOOSE THE RIGHT MODEL

In [ ]:
model = keras.models.load_model(new_path_model + 'model_nn_'+timetag+'.h5')

In [ ]:
y_out_norm = model.predict(x_val_arr)

In [ ]:
y_out = (y_out_norm * normalisation_coeff['melt_m_ice_per_y'].loc['x_range']) + normalisation_coeff['melt_m_ice_per_y'].loc['x_mean']

In [ ]:
y_out_pd_s = pd.Series(y_out[:,0],index=df_nrun.index,name='predicted_melt') 
y_target_pd_s = pd.Series(y_val_arr,index=df_nrun.index,name='reference_melt') 

In [ ]:
y_out_xr = y_out_pd_s.to_xarray()
y_target_xr = y_target_pd_s.to_xarray()
y_to_compare = xr.merge([y_out_xr.T, y_target_xr.T]).sortby('y')

In [ ]:
y_to_compare.to_netcdf(new_path_output+'NN_melt_predicted_reference_m_ice_per_yr_'+nemo_run+'.nc')